In [4]:
import pandas as pd
data = pd.read_csv('/content/Alphabets_data.csv')

print(data.head())
print(data.info())


  letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0      10       8      0       8      0       8  
1       3       9      2       8      4      10  
2       3       7      3       7      3       9  
3       4      10      6      10      2       8  
4       5       9      1       7      5      10  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbo

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = data.drop(columns='letter')
y = data['letter']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape}, Test set size: {X_test.shape}")


Training set size: (16000, 16), Test set size: (4000, 16)


In [7]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


X = data.drop(columns='letter')
y = data['letter']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Hidden layer 1
model.add(Dense(32, activation='relu'))  # Hidden layer 2
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output layer (size based on number of unique classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train_cat, epochs=50, batch_size=32, validation_data=(X_test, y_test_cat))

loss, accuracy = model.evaluate(X_test, y_test_cat)
print(f"Test accuracy: {accuracy * 100:.2f}%")



Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1429 - loss: 3.0438 - val_accuracy: 0.4322 - val_loss: 2.0015
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4789 - loss: 1.8534 - val_accuracy: 0.5683 - val_loss: 1.5207
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5941 - loss: 1.4690 - val_accuracy: 0.6463 - val_loss: 1.2943
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6514 - loss: 1.2666 - val_accuracy: 0.6895 - val_loss: 1.1622
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6860 - loss: 1.1615 - val_accuracy: 0.7075 - val_loss: 1.0767
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7081 - loss: 1.0814 - val_accuracy: 0.7278 - val_loss: 1.0281
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7204 - loss: 1.0153 - val_accuracy: 0.7340 - val_loss: 0.9774
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7334 - loss: 0.9857 - val_accuracy: 0.7535 - val_

In [9]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
def create_model(neurons=64, activation='relu'):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(32, activation=activation))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
param_grid = {
    'neurons': [32, 64, 128],
    'activation': ['relu', 'tanh'],
}
def grid_search():
    best_score = 0
    best_params = None

    for neurons in param_grid['neurons']:
        for activation in param_grid['activation']:
            model = create_model(neurons=neurons, activation=activation)
            history = model.fit(X_train, y_train_cat, epochs=50, batch_size=32, validation_data=(X_test, y_test_cat), verbose=0)
            score = model.evaluate(X_test, y_test_cat, verbose=0)[1]  # Use accuracy as score
            if score > best_score:
                best_score = score
                best_params = {'neurons': neurons, 'activation': activation}

    return best_params, best_score
best_params, best_score = grid_search()
print(f"Best Hyperparameters: {best_params} with accuracy {best_score:.4f}")


Best Hyperparameters: {'neurons': 64, 'activation': 'tanh'} with accuracy 0.8867


In [10]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
print(classification_report(y_test, y_pred_classes))

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
              precision    recall  f1-score   support

           0       0.79      0.95      0.86       149
           1       0.89      0.82      0.85       153
           2       0.94      0.80      0.87       137
           3       0.84      0.90      0.87       156
           4       0.89      0.80      0.84       141
           5       0.79      0.89      0.84       140
           6       0.80      0.81      0.80       160
           7       0.78      0.67      0.72       144
           8       0.91      0.85      0.88       146
           9       0.89      0.89      0.89       149
          10       0.73      0.88      0.80       130
          11       0.87      0.87      0.87       155
          12       0.93      0.93      0.93       168
          13       0.90      0.87      0.89       151
          14       0.79      0.86      0.82       145
          15       0.97      0.81      0.88       173
          16       0.86      0.88      0